# Herramientas Clásicas de Procesamiento de Lenguaje Natural

Este notebook contiene material teórico y ejemplos sobre herramientas clásicas de procesamiento de lenguaje natural.

In [9]:
!pip install -U nltk spacy scikit-learn

  Using cached spacy-3.8.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (27 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.2/774.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 1.0.1
    Uninstalling blis-1.0.1:
      Successfully uninstalled blis-1.0.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.2
    Uninstalling thinc-8.3.2:
      Successfully uninstalled thinc-8.3.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.2
    Uninstalling spacy-3.8.2:
      Successfully uninstalled spacy-3.8.2


In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# Alternativa usando ToktokTokenizer
tokenizer = nltk.tokenize.ToktokTokenizer()
tokens = tokenizer.tokenize(texto)
print("Tokens ToktokTokenizer:", tokens)

Tokens ToktokTokenizer: ['¡', 'Hola', 'mundo', '!', 'Esta', 'es', 'una', 'prueba', '.']


In [12]:
import sys
!{sys.executable} -m pip install -U nltk spacy sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [13]:
import nltk
import spacy
import sklearn

spacy.cli.download("en_core_web_sm")
spacy.cli.download("es_core_news_sm")
spacy.cli.download("es_core_news_md")

def test_spacy_model(model_name):
    try:
        spacy.load(model_name)
        print(f"Model {model_name} already installed.")
    except OSError:   
        print(f"Model {model_name} not found. Installing...")
        spacy.cli.download(model_name)
        print("Installation sucessful.")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/exp

In [14]:
test_spacy_model("es_core_news_sm")

Model es_core_news_sm already installed.


En este notebook vamos a repasar herramientas clásicas (usadas en modelos de machine learning, aunque menos en *deep learning*) que pueden sernos útiles para trabajar con texto, y que nos van a permitir dar solución a muchas de las peculiaridades del lenguaje natural.


<div class="alert alert-block alert-warning">
<b>Pregunta:</b> 
Un texto es una unidad compleja, que se puede dividir en párrafos, frases, palabras, sílabas o incluso caracteres. ¿A qué nivel crees que tratan los modelos de ML el texto? ¿Cuáles crees que son las ventajas e inconvenientes de trabajar a más alto o bajo nivel?
</div>


## Simplificación de los datos

Uno de los principales problemas de las representaciones clásicas de texto es la dimensionalidad, ya que nos basamos en un diccionario, que fácilmente puede tener $10-100$ K tokens. Así, surge un problema técnico que nos lleva a intentar reducir la dimensionalidad de los vectores utilizados. 

Podemos ver un texto como una unidad de comunicación humana bastante compleja. Un documento, hasta el más pequeño, está hecho no sólo de palabras, sino de un sinnúmero de relaciones semánticas que solo pueden ser descodificadas por quienes dominan ciertos códigos. En fin, que son un desastre y un dolor de cabeza para quienes están acostumbrados a la información estructurada (e.g., en tablas o esquemas).

Extraer automáticamente información relevante de los textos es una tarea trivial para un ser humano, pero un verdadero reto para una máquina. Muchas veces no nos interesa conocer todos los significados de un texto, sino solamente algunos pertinentes para realizar una tarea. Aunque las computadoras (aún) no entienden el lenguaje natural, son muy competentes leyendo superficialmente grandes cantidades de texto en segundos.

Una buena técnica para obtener la información relevante de un texto consiste en eliminar los elementos que puedan ser irrelevantes, y resaltar más lo que de verdad contengan información. Las siguientes herramientas tienen el mismo objetivo, y es el de atacar el problema de la dimensionalidad, intentando simplificar a las máquinas el texto libre que reciben.

### tokenización

Vamos a eliminar las palabras que tienen poco interés para nosotros. El primer paso es delimitar las palabras del texto, y convertir esas palabras en elementos de una lista, los **tokens**. Este procedimiento es conocido como tokenización. Este proceso también está repleto de ambigüedades:
* hispano-romano, astur-leonés
* aren't, o'neill
* whitespace, white space, 


Hay diversas librerías que nos facilitan la tokenización, y este proceso se puede hacer usando reglas, o mediante modelos estadísticos. Dos librerías que facilitan esta tarea son `spacy` y `nltk`. O se puede hacer directamente en Python si la regla es sencilla, incluso usando expresiones regulares, tema que trataremos en la siguiente sesión.

In [15]:
texto = '''My name isn't David. El producto cuesta 3€. Este procedimiento es conocido como tokenización. Vamos a implementar esto en Python. Lo ha hecho la empresa TEST S.A. '''

In [16]:
texto.split()

['My',
 'name',
 "isn't",
 'David.',
 'El',
 'producto',
 'cuesta',
 '3€.',
 'Este',
 'procedimiento',
 'es',
 'conocido',
 'como',
 'tokenización.',
 'Vamos',
 'a',
 'implementar',
 'esto',
 'en',
 'Python.',
 'Lo',
 'ha',
 'hecho',
 'la',
 'empresa',
 'TEST',
 'S.A.']

La librería `nltk` tiene un amplio rango de [tokenizers](https://www.nltk.org/api/nltk.tokenize.html), tanto para palabras, como para frases y sílabas. Incluso hay uno específico para tweets `nltk.tokenize.TweetTokenizer()`.

In [17]:
tokenizer = nltk.tokenize.ToktokTokenizer()
tokenizer.tokenize(texto)

['My',
 'name',
 'isn',
 "'",
 't',
 'David.',
 'El',
 'producto',
 'cuesta',
 '3€',
 '.',
 'Este',
 'procedimiento',
 'es',
 'conocido',
 'como',
 'tokenización.',
 'Vamos',
 'a',
 'implementar',
 'esto',
 'en',
 'Python.',
 'Lo',
 'ha',
 'hecho',
 'la',
 'empresa',
 'TEST',
 'S.A.']

In [18]:
tokenizer = nltk.tokenize.NLTKWordTokenizer()
tokenizer.tokenize(texto)

['My',
 'name',
 'is',
 "n't",
 'David.',
 'El',
 'producto',
 'cuesta',
 '3€.',
 'Este',
 'procedimiento',
 'es',
 'conocido',
 'como',
 'tokenización.',
 'Vamos',
 'a',
 'implementar',
 'esto',
 'en',
 'Python.',
 'Lo',
 'ha',
 'hecho',
 'la',
 'empresa',
 'TEST',
 'S.A',
 '.']

El recomendado por `nltk`, y que permite elegir el idioma, es `nltk.tokenize.word_tokenize()`:

In [19]:
nltk.download('punkt') # descarga los archivos necesarios
#tokenize = lambda x: nltk.tokenize.word_tokenize(x, language='spanish')
#tokenize(texto)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/odremanferrer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
dir(nltk.tokenize)

['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer',
 'PunktSentenceTokenizer',
 'PunktTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'SyllableTokenizer',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordDetokenizer',
 'TreebankWordTokenizer',
 'TweetTokenizer',
 'WhitespaceTokenizer',
 'WordPunctTokenizer',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_get_punkt_tokenizer',
 '_treebank_word_tokenizer',
 'api',
 'blankline_tokenize',
 'casual',
 'casual_tokenize',
 'destructive',
 'functools',
 'legality_principle',
 'line_tokenize',
 'load',
 'mwe',
 'punkt',
 're',
 'regexp',
 'regexp_span_tokenize',
 'regexp_tokenize',
 'repp',
 'sent_tokenize',
 'sexpr',
 'sexpr_tokenize',
 'simple',
 'sonority_sequencing',
 'stanford_segmenter',
 'string_span_tokenize',


En `spacy`, además de la librería, debes descargar el modelo de la lengua que vas a utilizar. 

Para descargar el modelo del español, por ejemplo, escribe en tu terminal lo siguiente: 

`python -m spacy download es`


In [21]:
import spacy

test_spacy_model('es_core_news_sm')
nlp = spacy.load('es_core_news_sm') # Crea un objeto de spacy tipo nlp, que tokeniza directamente

Model es_core_news_sm already installed.


Si aplicamos el modelo de lenguaje `nlp` al texto, directamente lo tokeniza, pero además aplica PoS tagging y NER por defecto:

In [22]:
doc = nlp(texto) 
tokens = [t.text for t in doc] # Crea una lista con las palabras del texto
tokens

['My',
 'name',
 "isn't",
 'David',
 '.',
 'El',
 'producto',
 'cuesta',
 '3€.',
 'Este',
 'procedimiento',
 'es',
 'conocido',
 'como',
 'tokenización',
 '.',
 'Vamos',
 'a',
 'implementar',
 'esto',
 'en',
 'Python',
 '.',
 'Lo',
 'ha',
 'hecho',
 'la',
 'empresa',
 'TEST',
 'S.A.']

Si lo **único que queremos es tokenizar, es mucho más eficiente** decirle a spacy que sólo haga eso, o crear un tokenizador a partir de `nlp`. Se puede hacer de muchas formas:

In [23]:
with nlp.disable_pipes(nlp.pipe_names): # desactivamos todo
    doc = nlp(texto)
print([tok.text for tok in doc])

['My', 'name', "isn't", 'David', '.', 'El', 'producto', 'cuesta', '3€.', 'Este', 'procedimiento', 'es', 'conocido', 'como', 'tokenización', '.', 'Vamos', 'a', 'implementar', 'esto', 'en', 'Python', '.', 'Lo', 'ha', 'hecho', 'la', 'empresa', 'TEST', 'S.A.']


In [24]:
doc = list(nlp.pipe([texto],disable=nlp.pipe_names))[0]
print([tok.text for tok in doc])

['My', 'name', "isn't", 'David', '.', 'El', 'producto', 'cuesta', '3€.', 'Este', 'procedimiento', 'es', 'conocido', 'como', 'tokenización', '.', 'Vamos', 'a', 'implementar', 'esto', 'en', 'Python', '.', 'Lo', 'ha', 'hecho', 'la', 'empresa', 'TEST', 'S.A.']


In [25]:
dir(nlp)

['Defaults',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_components',
 '_config',
 '_disabled',
 '_ensure_doc',
 '_ensure_doc_with_context',
 '_factory_meta',
 '_get_pipe_index',
 '_has_gpu_model',
 '_link_components',
 '_meta',
 '_multiprocessing_pipe',
 '_optimizer',
 '_path',
 '_pipe_configs',
 '_pipe_meta',
 '_resolve_component_status',
 'add_pipe',
 'analyze_pipes',
 'batch_size',
 'begin_training',
 'component',
 'component_names',
 'components',
 'config',
 'create_optimizer',
 'create_pipe',
 'create_pipe_from_source',
 'default_config',
 'default_error_handler',
 'disable_pipe',
 'disable_pipes',
 'disabl

In [26]:
nlp = spacy.load('es_core_news_sm') # Crea un objeto de spacy tipo nlp, que tokeniza directamente

In [27]:
tokenizer = nlp.tokenizer
tokens = [t.text for t in tokenizer(texto)] # Crea una lista con las palabras del texto
tokens

['My',
 'name',
 "isn't",
 'David',
 '.',
 'El',
 'producto',
 'cuesta',
 '3€.',
 'Este',
 'procedimiento',
 'es',
 'conocido',
 'como',
 'tokenización',
 '.',
 'Vamos',
 'a',
 'implementar',
 'esto',
 'en',
 'Python',
 '.',
 'Lo',
 'ha',
 'hecho',
 'la',
 'empresa',
 'TEST',
 'S.A.']

In [28]:
list(nlp.tokenizer(texto))

[My,
 name,
 isn't,
 David,
 .,
 El,
 producto,
 cuesta,
 3€.,
 Este,
 procedimiento,
 es,
 conocido,
 como,
 tokenización,
 .,
 Vamos,
 a,
 implementar,
 esto,
 en,
 Python,
 .,
 Lo,
 ha,
 hecho,
 la,
 empresa,
 TEST,
 S.A.]

### Stop words

Son palabras excesivamente comunes que no añaden significado a un texto, debido al gran tamaño de los vocabularios y por ende, de los vectores que codificaban texto (*sparsity*), se eliminaban directamente. La eliminación de las conocidas como **stop words** no es obligatoria, y su utilización ha disminuido recientemente. Queda muy bien explicado en el siguiente fragmento de [Intro to Information Retrieval](https://nlp.stanford.edu/IR-book/html/htmledition/dropping-common-terms-stop-words-1.html):


    Some extremely common words which would appear to be of little value in helping select documents matching a user need are excluded from the vocabulary entirely. These words are called stop words.

    The general trend in IR systems over time has been from standard use of quite large stop lists (200-300 terms) to very small stop lists (7-12 terms) to no stop list whatsoever. Web search engines generally do not use stop lists.

Las listas de *stop words* no son únicas, y diferentes librerías tienen listas distintas, como veremos a continuación. Además, lo que es y no es una stop word **depende del problema**, ya que una misma palabra puede tener importancia en un contexto, pero no en otro.

Además de las stop words, también es común eliminar la **puntuación** y/o **símbolos** `@,$,€,#` que creemos que no nos van a aportar significado.

`sklearn` también tiene su lista de stop words en inglés:

In [29]:
sk_stopwords = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS
print(sorted(list(sk_stopwords))[:20])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst']


Vamos ahora a las librerías especializadas en NLP, como `nltk`:

In [30]:
#from nltk.corpus import stopwords as nltk_stopwords
#print(sorted(nltk_stopwords.words('english'))[:20])

`nltk` tiene listas de stop words en varios idiomas:

In [31]:
#stopwords_sp_nltk = sorted(nltk_stopwords.words('spanish'))
#print(stopwords_sp_nltk[:20])

Podemos hacer lo mismo con `spacy`:

In [32]:
print(sorted(spacy.lang.es.stop_words.STOP_WORDS)[:20])

['a', 'acuerdo', 'adelante', 'ademas', 'además', 'afirmó', 'agregó', 'ahi', 'ahora', 'ahí', 'al', 'algo', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'alli', 'allí', 'alrededor']


<div class="alert alert-block alert-warning">
<b>Ejercicio:</b> 
Comprueba si palabras como <code>['nada','no','mal','bien','bueno','ninguno']</code> son <i>stop words</i>. ¿Tiene esto sentido?
</div>

In [33]:
stopwords_spacy = spacy.lang.es.stop_words.STOP_WORDS
"hola" in stopwords_spacy

False

In [34]:
#len(nltk_stopwords.words('spanish'))

In [35]:
len(stopwords_spacy)

521

In [36]:
#stopwords_sp = sorted(spacy.lang.es.stop_words.STOP_WORDS)
#for word in ['nada','no','mal','bien','bueno','ninguno']:
#    for lib, sw in zip(['nltk','spacy'],[stopwords_sp_nltk, stopwords_sp]):
#        print(f"'{word}' is stop word in {lib}: {word in sw}")

<div class="alert alert-block alert-warning">
<b>Ejercicio:</b> 
    Calcula los tokens que estén en la lista de <i>stop words</i> de <code>spacy</code>, pero no en la de <code>nltk</code>. También la longitud de cada conjunto. Por ejemplo, para el idioma español.
</div>

In [37]:
#set(stopwords_sp)- set(stopwords_sp_nltk) 

In [38]:
#set(stopwords_sp_nltk) - set(stopwords_sp)

<div class="alert alert-block alert-warning">
<b>Ejercicio:</b>
¿En qué tareas de NLP la eliminación de <i>stop words</i> no es viable?
</div>

Algunas tareas donde no es aplicable porque se pierde información esencial serían:

* Traducción automática
* Modelado del lenguage (Language Modeling)
* Resumen automático de textos
* Sistemas conversacionales o de Pregunta Respuesta

### Stemming y Lemmatization

En general, podemos observar que hay palabras diferentes que representan significados parecidos. En español, por ejemplo, sabemos que *canto, cantas, canta, cantamos, cantáis, cantan* son distintas formas (conjugaciones) de un mismo verbo (*cantar*). Y que *niña, niño, niñita, niños, niñotes, y otras más*, son distintas formas del vocablo *niño*. Así que sería genial poder obviar las diferencias y juntar todas estas variantes en un mismo token.
Stemming y Lemmatization son dos transformaciones parecidas, que nos devuelven el lema o raiz de la palabra.

* **Lematización**: relaciona una palabra flexionada o derivada con su forma **canónica o lema**. Y un lema no es otra cosa que la **forma que tienen las palabras cuando las buscas en el diccionario**. La lematización tiene dos costes. Primero, es un proceso que consume recursos (sobre todo tiempo). Segundo, suele ser probabilística, así que en algunos casos obtendremos resultados inesperados.


* **Radicalización** o **stemming**: procedimiento de convertir palabras en raíces. Estas raíces son la parte invariable de palabras relacionadas sobre todo por su forma. De cierta manera se parece a la lematización, pero los resultados (las raíces) no tienen por qué ser palabras de un idioma. Por ejemplo, el algoritmo de stemming puede decidir que la raíz de *amamos* no es *am-* sino *amam-*. El stemming es mucho más rápido, ya que es un procedimiento heurístico que recorta el final de las palabras.

"Stemming is the poor-man’s lemmatization." (Noah Smith, 2011)

**Es importante notar que ambos procesos dependen altamente del idioma.**

Para lematizar es importante la etiqueta *Part of Speech* (POS), es decir, si la palabra es un sustantivo, verbo, etc. En el caso de `nltk`, hay que indicar en cada caso esta etiqueta:

In [39]:
# nltk.download('wordnet')

In [40]:
word_list = ['feet', 'foot', 'foots', 'footing']

In [41]:
#wnl = nltk.stem.WordNetLemmatizer()
#[wnl.lemmatize(word, pos='n') for word in word_list]

Si queremos un lematizador en español, tenemos que ir a `spacy`, que además de manera automática aplica un **PoS-tagger** y hace muy sencillo obtener el *lema*:

In [42]:
nlp = spacy.load('es_core_news_sm')

In [43]:
doc = nlp("Ella se compró el coche más caro de la marca.")
lemmas = [tok.lemma_.lower() for tok in doc]
for tok in doc:
    print(tok.text, '->', tok.lemma_.lower())

Ella -> él
se -> él
compró -> comprar
el -> el
coche -> coche
más -> más
caro -> caro
de -> de
la -> el
marca -> marca
. -> .


In [44]:
doc = nlp("Él se compró el coche más caro de la marca.")
lemmas = [tok.lemma_.lower() for tok in doc]
for tok in doc:
    print(tok.text, '->', tok.lemma_.lower())

Él -> él
se -> él
compró -> comprar
el -> el
coche -> coche
más -> más
caro -> caro
de -> de
la -> el
marca -> marca
. -> .


In [45]:
doc = nlp(texto)
lemmas = [tok.lemma_.lower() for tok in doc]
for tok in doc:
    print(tok.text, '->', tok.lemma_.lower())

My -> my
name -> namir
isn't -> isn't
David -> david
. -> .
El -> el
producto -> producto
cuesta -> costar
3€. -> 3€.
Este -> este
procedimiento -> procedimiento
es -> ser
conocido -> conocer
como -> como
tokenización -> tokenización
. -> .
Vamos -> ir
a -> a
implementar -> implementar
esto -> este
en -> en
Python -> python
. -> .
Lo -> él
ha -> haber
hecho -> hacer
la -> el
empresa -> empresa
TEST -> test
S.A. -> s.a.


Aunque te parezca sorprendente, `spacy` no contiene ninguna función para stemming, ya que se basa únicamente en la lematización. Por lo tanto, en esta sección, utilizaremos únicamente `NLTK`.


Hay dos tipos de stemmers en NLTK: [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/) and [Snowball stemmers](https://tartarus.org/martin/PorterStemmer/). Cada uno de ellos se ha implementado siguiendo algoritmos distintos. Snowball stemmer es una versión ligeramente mejorada del Porter stemmer y generalmente se prefiere sobre este último, además, **sólo Snowball stemmer funciona para idiomas distintos al inglés**.

In [46]:
sp_snowball = nltk.SnowballStemmer('spanish')
tokens = texto.split()  # crear una lista de tokens
stems = [sp_snowball.stem(token) for token in tokens]
for w, s in zip(tokens, stems):
    print(w, '->', s)

My -> my
name -> nam
isn't -> isn't
David. -> david.
El -> el
producto -> product
cuesta -> cuest
3€. -> 3€.
Este -> este
procedimiento -> proced
es -> es
conocido -> conoc
como -> com
tokenización. -> tokenizacion.
Vamos -> vam
a -> a
implementar -> implement
esto -> esto
en -> en
Python. -> python.
Lo -> lo
ha -> ha
hecho -> hech
la -> la
empresa -> empres
TEST -> test
S.A. -> s.a.


<div class="alert alert-block alert-warning">
<b>Ejercicio:</b> 
    Prueba a lematizar y radicalizar las siguientes palabras:

* Inglés: fly, flies, flying
* Español: universo, universidad, universal
</div>

In [74]:
#wnl = nltk.stem.WordNetLemmatizer()
#[wnl.lemmatize(word, pos='n') for word in ["fly", "flies", "flying"]]

['universo', 'universidad', 'universal']

In [75]:
doc = nlp("universo, universidad, universal")
lemmas = [tok.lemma_.lower() for tok in doc]
for tok in doc:
    print(tok.text, '->', tok.lemma_.lower())

universo -> universo
, -> ,
universidad -> universidad
, -> ,
universal -> universal


In [49]:
sp_snowball = nltk.SnowballStemmer('spanish')
tokens = "universo, universidad, universal".split(", ")  # crear una lista de tokens
stems = [sp_snowball.stem(token) for token in tokens]
for w, s in zip(tokens, stems):
    print(w, '->', s)

universo -> univers
universidad -> univers
universal -> universal


### spaCy vs NLTK

La librería spaCy es una de las más populares en NLP junto con NLTK. La diferencia básica entre las dos es el hecho de que **NLTK contiene una amplia variedad de algoritmos** para resolver un problema, mientras que **spaCy contiene solo uno, el mejor**.

NLTK se lanzó en 2001, mientras que spaCy es relativamente nuevo y se desarrolló en 2015. En este curso sobre NLP, trataremos principalmente de spaCy, debido a que en general es **state of the art**. Sin embargo, también usaremos NLTK cuando sea más fácil realizar una tarea que con spaCy.

**Si puedes hacerlo en spaCy, mejor usar spacy.** Además, la documentación de spaCy es mucho más intuitiva.

### ¿Cuándo usar estos métodos?

Estas técnicas fueron consideradas técnicas estándar durante mucho tiempo, pero a menudo pueden **dañar** el rendimiento **si se utiliza deep learning**. El stemming, la lematización y la eliminación *stop words* implican una pérdida de información en muchos casos.

Sin embargo, aún pueden ser útiles cuando se trabaja con modelos más simples. A grandes rasgos, estamos simplificando los datos, lo que es una técnica de **regularización**, es decir, evitamos el **overfitting** a features que sólo nos generan ruido. 

Como siempre en el Machine Learning, no es fácil situar la línea entre el ruido y la señal. Sólo la experiencia, y el sobre todo tiempo para validar entre distintas opciones deben ser la guía que nos indique qué usar en el modelo final.


<img src="resources/skomoroch.png" alt="" style="width: 65%"/>

## Vectorizando texto

Hasta ahora hemos estudiado formas de reducir la complejidad del texto, todas ellas ayudándonos a reducir el tamaño del vocabulario (conjunto de palabras/tokens únicos) de nuestros modelos. Los algoritmos de **Machine Learning** trabajan con datos numéricos, no pueden usar el texto directamente. Hay muchas formas de convertir el texto en vectores numéricos (**feature engineering**), y vamos a explorar las dos formas más básicas.


### Bag of words

Para crear la representación de *bag of words*, cada token es una dimensión en un espacio de dimensión el tamaño del vocabulario $V$. Para construir el vector de una frase, será la suma de los vectores de cada token, es decir, convertimos cada frase en un vector que cuenta el número de ocurrencias de cada token. Hacemos la distinción entre palabra y token porque **se pueden usar como tokens n-gramas de palabras**, lo que aumenta la complejidad del modelo, pero permite captar correlaciones en el orden de las palabras.


Se siguien los pasos:
1. Encuentra los **V** tokens mas comunes del corpus de entrenamiento y se les asigna un índice, este es nuestro **vocabulario**. Creamos un diccionario para convertir de tokens a índices y viceversa.
2. Para cada frase en el corpus, creamos un vector de dimensión **V** y lo inicializamos con ceros.
3. Iteramos sobre los tokens de cada frase, y si el token está en el diccionario, incrementamos en 1 el índice correspondiente del vector.

Veamos el siguiente ejemplo con **V=4**, cuyo vocabulario es:

    ['hi', 'you', 'me', 'are']

Les asignamos un índice: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

Y si tenemos la siguiente frase:

    'hi, how are you? are you ok?'

que hemos preprocesado a la siguiente lista de tokens:

    ['hi', 'how', 'are', 'you', 'are', 'you', 'ok']

Inicializamos el vector:

    [0, 0, 0, 0]
    
Iteramos sobre las palabras, teniendo en cuenta sólo aquellas en nuestro vocabulario:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]
    'are': [1, 1, 0, 2]
    'you': [1, 2, 0, 2]
    'ok':  [1, 2, 0, 2] # word 'ok' is not in our dictionary

El vector resultante es:

    [1, 2, 0, 2]
    
En el [s1_challenge](./s1_challenge.ipynb) construiremos este vectorizador a mano, pero viene implementado en sklearn en la función [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

<div class="alert alert-block alert-warning">
<b>Ejercicio:</b> 
    Usando el vocabulario del ejemplo anterior, cuál sería el vector <i>bag of words</i> de la frase con tokens

* <code>['hi', 'i', 'am', 'david,', 'and', 'you?', 'are', 'you', 'ok?']</code>
* <code>{'hi': 0, 'you': 1, 'me': 2, 'are': 3}</code>
</div>

In [50]:
[1, 1, 0, 1]

[1, 1, 0, 1]

<div class="alert alert-block alert-warning">
<b>Ejercicio:</b> 
    La variable <code>quijote</code> contiene el texto completo del Quijote:
    
* ¿De qué tamaño serán los vectores <i>bag of words</i> sin utilizar ninguna de las técnicas de reducción anteriores? Tokeniza con <code>str.split()</code> y con el tokenizador de una de las librerías.
* ¿Por qué si tokenizamos con la librería cambia tanto el tamaño del vocabulario?
* ¿Y si sólo trabajas con minúsculas?
</div>

In [51]:
with open('./resources/quijote_largo.txt', 'r', encoding='UTF-8') as f:
    quijote = f.read()

In [52]:
len(set(quijote.split()))

22938

In [53]:
tokenizer = nlp.tokenizer
tokens = [t.text for t in tokenizer(quijote)] # Crea una lista con las palabras del texto
len(set(tokens))

15615

In [76]:
tokens

['don',
 'quijote',
 'de',
 'la',
 'mancha',
 '\n',
 'miguel',
 'de',
 'cervantes',
 'saavedra',
 '\n\n',
 'primera',
 'parte',
 '\n',
 'capítulo',
 '1',
 ':',
 'que',
 'trata',
 'de',
 'la',
 'condición',
 'y',
 'ejercicio',
 'del',
 'famoso',
 'hidalgo',
 'd.',
 'quijote',
 'de',
 'la',
 'mancha',
 '\n',
 'en',
 'un',
 'lugar',
 'de',
 'la',
 'mancha',
 ',',
 'de',
 'cuyo',
 'nombre',
 'no',
 'quiero',
 'acordarme',
 ',',
 'no',
 'ha',
 'mucho',
 'tiempo',
 'que',
 'vivía',
 'un',
 'hidalgo',
 'de',
 'los',
 'de',
 'lanza',
 'en',
 'astillero',
 ',',
 'adarga',
 'antigua',
 ',',
 'rocín',
 'flaco',
 'y',
 'galgo',
 'corredor',
 '.',
 'una',
 'olla',
 'de',
 'algo',
 'más',
 'vaca',
 'que',
 'carnero',
 ',',
 'salpicón',
 'las',
 'más',
 'noches',
 ',',
 'duelos',
 'y',
 'quebrantos',
 'los',
 'sábados',
 ',',
 'lentejas',
 'los',
 'viernes',
 ',',
 'algún',
 'palomino',
 'de',
 'añadidura',
 'los',
 'domingos',
 ',',
 'consumían',
 'las',
 'tres',
 'partes',
 'de',
 'su',


In [78]:
len(set(quijote.split())) # 22,938 tokens únicos
tokenizer = nlp.tokenizer
tokens = [t.text for t in tokenizer(quijote)]
len(set(tokens)) # 15,615 tokens únicos
len(set(quijote.lower().split())) # 22,211 tokens únicos

22211

In [55]:
# tamaño del vocabulario

len(set(quijote.split()))

22938

In [56]:
# esto con minusculas
len(set(quijote.lower().split()))

22211

In [57]:
quijote.split()[:200]

['DON',
 'QUIJOTE',
 'DE',
 'LA',
 'MANCHA',
 'Miguel',
 'de',
 'Cervantes',
 'Saavedra',
 'PRIMERA',
 'PARTE',
 'CAPÍTULO',
 '1:',
 'Que',
 'trata',
 'de',
 'la',
 'condición',
 'y',
 'ejercicio',
 'del',
 'famoso',
 'hidalgo',
 'D.',
 'Quijote',
 'de',
 'la',
 'Mancha',
 'En',
 'un',
 'lugar',
 'de',
 'la',
 'Mancha,',
 'de',
 'cuyo',
 'nombre',
 'no',
 'quiero',
 'acordarme,',
 'no',
 'ha',
 'mucho',
 'tiempo',
 'que',
 'vivía',
 'un',
 'hidalgo',
 'de',
 'los',
 'de',
 'lanza',
 'en',
 'astillero,',
 'adarga',
 'antigua,',
 'rocín',
 'flaco',
 'y',
 'galgo',
 'corredor.',
 'Una',
 'olla',
 'de',
 'algo',
 'más',
 'vaca',
 'que',
 'carnero,',
 'salpicón',
 'las',
 'más',
 'noches,',
 'duelos',
 'y',
 'quebrantos',
 'los',
 'sábados,',
 'lentejas',
 'los',
 'viernes,',
 'algún',
 'palomino',
 'de',
 'añadidura',
 'los',
 'domingos,',
 'consumían',
 'las',
 'tres',
 'partes',
 'de',
 'su',
 'hacienda.',
 'El',
 'resto',
 'della',
 'concluían',
 'sayo',
 'de',
 'velarte,',


In [58]:
#tokenize(quijote)[:200]

In [59]:
#tokenize = lambda x: nltk.tokenize.word_tokenize(x, language='spanish')
#len(set(tokenize(quijote)))

In [60]:
#len(set(tokenize(quijote.lower())))

In [61]:
print(f'Python split, V: {len(set(quijote.split()))}')

Python split, V: 22938


In [62]:
print(f'Python split, V: {len(set(quijote.lower().split()))}')

Python split, V: 22211


In [63]:
sorted(set(quijote.lower().split()))[:20]

['"apenas',
 '"caballero',
 '"conde',
 '"dichosa',
 '"donde',
 '"el',
 '"más',
 '"miau",',
 '"no',
 '"rastrea',
 '"ricamonte",',
 '"tablante",',
 '"y',
 '"¡oh,',
 "''éste",
 "''¡ea,",
 '(a',
 '(al',
 '(como',
 '(con']

In [64]:
nlp = spacy.load("es_core_news_sm")
doc = nlp.tokenizer(quijote.lower().replace("-", " "))
tokens = [t.text for t in doc]

In [65]:
sorted(set(tokens))[:20]

['\n',
 '\n\n',
 '\n ',
 '\n  ',
 ' ',
 '  ',
 '!',
 '"',
 "''",
 '(',
 ')',
 ',',
 '.',
 '...',
 '.luego',
 '1',
 '10',
 '11',
 '12',
 '13']

In [66]:
len(set(tokens))

14586

In [67]:
#print(
#    f'NLTK word_tokenize, V: {len(set(nltk.tokenize.word_tokenize(quijote.lower())))}')

In [68]:
# Crea un Tokenizer de la nada, sólo teniendo en cuenta el vocabulario
tokenizer_vocab = spacy.tokenizer.Tokenizer(nlp.vocab)
print(
    f'spacy tokenizer, V: { len({token.text for token in tokenizer_vocab(quijote)})} '
)

spacy tokenizer, V: 22940 


In [69]:
list(nlp.tokenizer("asdas sdasd sdas"))

[asdas, sdasd, sdas]

In [70]:
# Crea un Tokenizer con las opciones por defecto de un idioma
# en este caso incluye el vocabulario, puntuación y excepciones
tokenizer_complete = nlp.tokenizer
print(
    f'spacy tokenizer, V: { len({token.text for token in tokenizer_complete(quijote)})} '
)

spacy tokenizer, V: 15615 


In [71]:
''''
'print(f'Python split, V: {len(set(quijote.lower().split()))}')
print(
    f'NLTK word_tokenize, V: {len(set(nltk.tokenize.word_tokenize(quijote.lower())))}'
)
print(
    f'spacy tokenizer, V: {len({token.text for token in tokenizer_complete(quijote.lower())})} '
)'''

"'\n'print(f'Python split, V: {len(set(quijote.lower().split()))}')\nprint(\n    f'NLTK word_tokenize, V: {len(set(nltk.tokenize.word_tokenize(quijote.lower())))}'\n)\nprint(\n    f'spacy tokenizer, V: {len({token.text for token in tokenizer_complete(quijote.lower())})} '\n)"

### tf-idf (term frequency - inverse document frequency)

Una extensión sobre *bag of words* es la representación **tf-idf**. [Link](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). En este caso, también se cuenta la ocurrencia de cada token, pero se pondera para penalizar palabras que aparecen en casi todos los textos (como las *stop words*), por lo que no ayudan a diferenciarlos. Es muy popular para tareas de **information retrieval**.

Si utilizamos una representación *bag of words* para crear un modelo, esos términos muy frecuentes pueden ensombrecer otros tokens cuyas frecuencias son mucho menores, pero que contienen más información. Una posible solución a este problema es darle un peso distinto a cada una de las dimensiones del vector. El esquema de ponderado más común es:

* term frequency: 
$$
\operatorname{tf}(t, d) = \frac{|\{t \in d\}| }{|d|} \;,
$$
que es el porcentaje de veces que aparece el token $t$ en el documento $d$. Da la importancia que tiene un término en un documento concreto.
* inverse document frequency: 
$$
\operatorname{idf}(t, D)=\log \frac{|D|}{1+|\{d \in D: t \in d\}|} \;,
$$
donde $|D|$ es el número de documentos, y $|\{d \in D: t \in d\}|$ es el número de documentos en los que aparece el término $t$, el factor 1 se añade para que no se divida entre 0. Quiza importancia a términos que están presentes en muchos documentos.

Luego, cada término es ponderado por el siguiente factor:
$$
\operatorname{tfidf}(t, d, D) = \operatorname{tf}(t, d)\times \operatorname{idf}(t, D) \; .
$$

En el caso de `sklearn`, por defecto normaliza los vectores resultante según la norma euclídea o L2 ([Más información](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting)). La función que lo implementa es [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

<div class="alert alert-block alert-warning">
<b>Preguntas:</b> 
    
* ¿Qué distancia hay entre dos tokens en una representación *bag of words*? ¿Cambia esta distancia si los tokens tienen un significado más o menos cercano? ¿Cambia esto en la representación *tf-idf*?
* ¿En una tarea de clasificación, si un término aparece en todos los casos de una clase, cómo afecta al su valor de $\operatorname{tfidf}(t, d, D)$? ¿Es este término una feature importante para clasificar?
* Una vez contruido un vocabulario, ¿qué pasará si en el conjunto de validación, test o en producción aparecen palabras nuevas?
* ¿Cómo actúan las herramientas aprendidas ante las erratas? Analiza el caso de que existan erratas tanto durante el entrenamiento como en producción? Por ejemplo: 'ayuntamiento' vs 'ayuntameinto'.
* En el lenguaje, el orden de los factores altera el producto. 'Juan mordió a el perro' vs 'El perro mordió a Juan'. 'Si no viene a comer lo dejamos', 'No viene a comer si lo dejamos'. ¿Cómo actúa *bag of words* y *tf-idf* ante este fenómeno?
</div>

In [72]:
t1 = [1,0,0,0]

t2 = [0,1,0,0]

t3 = [0,0,1,0]